#### Date 30th july 2025 Project Rag for AMLGO LABS

In [1]:
# import main libraries 
import PyPDF2
import re
import json
import os
from typing import List
import nltk

# Download required NLTK data
nltk.download('punkt', quiet=True)
from nltk.tokenize import sent_tokenize

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# extract text from pdf
def extract_text_from_pdf(pdf_path: str) -> str:
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num, page in enumerate(pdf_reader.pages):
                text += page.extract_text()
                print(f"Processed page {page_num + 1}")
            return text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""
pdf_path = r"/home/petpooja/Documents/amlgo/data/AI Training Document.pdf"
raw_text = extract_text_from_pdf(pdf_path)
print(f"\n Document extracted")
print(f"Total characters: {len(raw_text):,}")
print(f"First 500 characters:\n{raw_text[:500]}...")

Processed page 1
Processed page 2
Processed page 3
Processed page 4
Processed page 5
Processed page 6
Processed page 7
Processed page 8
Processed page 9
Processed page 10
Processed page 11
Processed page 12
Processed page 13
Processed page 14
Processed page 15
Processed page 16
Processed page 17
Processed page 18
Processed page 19
Processed page 20

 Document extracted
Total characters: 68,669
First 500 characters:
User Agreement  
1. Introduction  
This User Agreement, the Mobile Application Terms of Use , and all policies and additional terms 
posted on and in our sites, applications, tools, and services (collectively "Services") set out the terms 
on which eBay offers you access to and use of our Services. You can find an overview of our policies 
here . The Mobile Application Terms of Use, all policies, and additional terms posted on and in our 
Services are incorporated into this User Agreement. You a...


In [3]:
# clean the text using Regullar expression
def clean_text(text: str) -> str:


    # Remove extra whitespace and normalize
    text = re.sub(r'\s+', ' ', text)


    
    # Remove special characters but keep essential punctuation
    text = re.sub(r'[^\w\s.,!?;:()\-\'\"]+', '', text)


    
    # Remove multiple consecutive punctuation
    text = re.sub(r'([.,!?;]){2,}', r'\1', text)
    

    return text.strip()

# Clean the extracted text
cleaned_text = clean_text(raw_text)
print(f"Text cleaned!")
print(f"Cleaned length: {len(cleaned_text):,} characters")
print(f"Reduction: {len(raw_text) - len(cleaned_text):,} characters removed")
print(f"\nCleaned sample:\n{cleaned_text[:500]}...")

Text cleaned!
Cleaned length: 67,504 characters
Reduction: 1,165 characters removed

Cleaned sample:
User Agreement 1. Introduction This User Agreement, the Mobile Application Terms of Use , and all policies and additional terms posted on and in our sites, applications, tools, and services (collectively "Services") set out the terms on which eBay offers you access to and use of our Services. You can find an overview of our policies here . The Mobile Application Terms of Use, all policies, and additional terms posted on and in our Services are incorporated into this User Agreement. You agree to ...


In [ ]:
## check it is working or not into my system 
import spacy
nlp = spacy.load("en_core_web_sm")

text = "Hello world. How are you? This is a test."
doc = nlp(text)

sentences = [sent.text for sent in doc.sents]
print(sentences)


['Hello world.', 'How are you?', 'This is a test.']


In [14]:
import spacy
from typing import List

nlp = spacy.load("en_core_web_sm")

# Intelligent chunking using spaCy
def sentence_aware_chunking(text: str, target_chunk_size: int = 200, overlap_sentences: int = 2) -> List[dict]:
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    
    chunks = []
    current_chunk = ""
    current_sentences = []
    
    for i, sentence in enumerate(sentences):
        test_chunk = current_chunk + " " + sentence if current_chunk else sentence
        
        if len(test_chunk) <= target_chunk_size or not current_chunk:
            current_chunk = test_chunk
            current_sentences.append(sentence)
        else:
            # Save current chunk
            if current_chunk:
                chunk_data = {
                    'id': len(chunks),
                    'text': current_chunk.strip(),
                    'length': len(current_chunk),
                    'sentence_count': len(current_sentences),
                    'start_sentence': i - len(current_sentences),
                    'end_sentence': i - 1
                }
                chunks.append(chunk_data)
            # Create new chunk with overlap
            if len(current_sentences) > overlap_sentences:
                overlap_text = " ".join(current_sentences[-overlap_sentences:])
                current_chunk = overlap_text + " " + sentence
                current_sentences = current_sentences[-overlap_sentences:] + [sentence]
            else:
                current_chunk = sentence
                current_sentences = [sentence]

    # Add final chunk
    if current_chunk:
        chunk_data = {
            'id': len(chunks),
            'text': current_chunk.strip(),
            'length': len(current_chunk),
            'sentence_count': len(current_sentences),
            'start_sentence': len(sentences) - len(current_sentences),
            'end_sentence': len(sentences) - 1
        }
        chunks.append(chunk_data)
    
    return chunks

# ---- RUN THE FUNCTION ----
print("Creating intelligent chunks")
chunks_data = sentence_aware_chunking(cleaned_text, target_chunk_size=250, overlap_sentences=2)

# Display statistics
total_chars = sum(chunk['length'] for chunk in chunks_data)
avg_length = total_chars / len(chunks_data)

print(f"\nChunking Results:")
print(f"Total chunks created: {len(chunks_data)}")
print(f"Average chunk length: {avg_length:.1f} characters")
print(f"Length range: {min(c['length'] for c in chunks_data)} - {max(c['length'] for c in chunks_data)} characters")

# Show sample chunks
print(f"\nSample chunks:")
for i in range(min(3, len(chunks_data))):
    chunk = chunks_data[i]
    print(f"\nChunk {i+1} (ID: {chunk['id']}):")
    print(f"Length: {chunk['length']} chars, Sentences: {chunk['sentence_count']}")
    print(f"Text: {chunk['text'][:200]}...")


Creating intelligent chunks

Chunking Results:
Total chunks created: 306
Average chunk length: 508.8 characters
Length range: 2 - 2131 characters

Sample chunks:

Chunk 1 (ID: 0):
Length: 17 chars, Sentences: 1
Text: User Agreement 1....

Chunk 2 (ID: 1):
Length: 275 chars, Sentences: 1
Text: Introduction This User Agreement, the Mobile Application Terms of Use , and all policies and additional terms posted on and in our sites, applications, tools, and services (collectively "Services") se...

Chunk 3 (ID: 2):
Length: 192 chars, Sentences: 2
Text: You can find an overview of our policies here . The Mobile Application Terms of Use, all policies, and additional terms posted on and in our Services are incorporated into this User Agreement....


In [17]:
# Save chunks with metadata
import pandas as pd
chunks_to_save = {
    'metadata': {
        'source_document': 'AI_Training_Document.pdf',
        'processing_date': str(pd.Timestamp.now()),
        'total_chunks': len(chunks_data),
        'avg_chunk_length': avg_length,
        'chunking_method': 'sentence_aware_with_overlap'
    },
    'chunks': chunks_data
}

# Ensure chunks directory exists
os.makedirs('../chunks', exist_ok=True)

# Save detailed chunk data
with open('../chunks/processed_chunks.json', 'w', encoding='utf-8') as f:
    json.dump(chunks_to_save, f, indent=2, ensure_ascii=False)

# Save simple chunk texts for embedding (backward compatibility)
chunk_texts = [chunk['text'] for chunk in chunks_data]
with open('../chunks/chunk_texts.json', 'w', encoding='utf-8') as f:
    json.dump(chunk_texts, f, indent=2, ensure_ascii=False)

print(" Chunks saved successfully!")
print(f"Files created:")
print(f"  - ../chunks/processed_chunks.json")
print(f"  - ../chunks/chunk_texts.json")
print(f"\n Document processing complete!")

 Chunks saved successfully!
Files created:
  - ../chunks/processed_chunks.json
  - ../chunks/chunk_texts.json

 Document processing complete!
